# World CO2 Emissions Forecasting
## LSTM Model
[*Cristian Castro Álvarez*](https://github.com/cristian-castro-a)

**Goal**: 
- Forecast the amount of World CO2 emissions (in tonnes) for the next 10 years (2021 - 2030).


**Data:**
- The data comes from [Our World in Data](https://github.com/owid/co2-data)
- Column 'CO2' of the dataframe indicates the Tonnes of CO2 emmitted into the atmosphere.
- The dataset includes yearly data from 1750 to 2020, with a total of 271 data points.

In [1]:
#Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow

import warnings
warnings.filterwarnings("ignore")

# Data